In [70]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
import sys

sys.path.append("../../")
import copy
import os
import logging
from src.utils import logging_utils
from src import functional
from src.models import ModelandTokenizer
# from src.data import load_relation
import json

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

In [72]:
RESULTS_DIR = os.path.join("../../", "results")
FIGURS_DIR = os.path.join("../../", "Figures")

MODEL = "pythia-2.8b-deduped" # "mamba-2.8b" | "pythia-2.8b-deduped"

causal_tracing_dir = os.path.join(RESULTS_DIR, "causal_tracing_aie", MODEL)

hooks = os.listdir(causal_tracing_dir)
hooks

['residual', 'mlp', 'attn']

In [73]:
from src.tracing import average_indirect_effects
from src.plotting import plot_trace_heatmap
from typing import Literal

# hook = hooks[0]
hook: Literal[
    "residual", 
    "ssm_after_ssm", "mlp_after_silu", "after_down_proj", # Mamba
    "mlp", "attn" # Pythia
    ] = "residual"

save_dir = os.path.join(FIGURS_DIR, "causal_tracing_aie", MODEL, hook)

relation_files = os.listdir(os.path.join(causal_tracing_dir, hook))

aie_all = {}

for relation_file in relation_files:

    with open(os.path.join(causal_tracing_dir, hook, relation_file), "r") as f:
        relation_ie_all = json.load(f)
    
    print(f"{relation_file} - {len(relation_ie_all.keys())} samples")

    for k, v in relation_ie_all.items():
        aie_all[f"{k}_{relation_file}_{hook}"] = v

    relation_aie = average_indirect_effects(relation_ie_all, relative_recovery = True)
    plot_trace_heatmap(
        relation_aie, 
        savepdf=os.path.join(save_dir, f"{relation_file}.pdf"),
        modelname="Mamba-2.8b"
    )

country_capital_city.json - 17 samples
2024-03-26 13:05:45 matplotlib.colorbar DEBUG    locator: <matplotlib.ticker.AutoLocator object at 0x7fd1f4470160>
2024-03-26 13:05:45 matplotlib.axes._base DEBUG    title position was updated manually, not adjusting


2024-03-26 13:05:45 matplotlib.axes._base DEBUG    title position was updated manually, not adjusting
2024-03-26 13:05:45 matplotlib.axes._base DEBUG    title position was updated manually, not adjusting
2024-03-26 13:05:45 matplotlib.backends.backend_pdf DEBUG    Assigning font /F1 = '/usr/share/fonts/truetype/msttcorefonts/Times_New_Roman.ttf'
2024-03-26 13:05:45 matplotlib.axes._base DEBUG    title position was updated manually, not adjusting
2024-03-26 13:05:45 matplotlib.backends.backend_pdf DEBUG    Embedding font /usr/share/fonts/truetype/msttcorefonts/Times_New_Roman.ttf.
2024-03-26 13:05:45 matplotlib.backends.backend_pdf DEBUG    Writing TrueType font.
2024-03-26 13:05:45 matplotlib.axes._base DEBUG    title position was updated manually, not adjusting
person_plays_pro_sport.json - 136 samples
2024-03-26 13:05:45 matplotlib.colorbar DEBUG    locator: <matplotlib.ticker.AutoLocator object at 0x7fd1e93251e0>
2024-03-26 13:05:45 matplotlib.axes._base DEBUG    title position was 

In [74]:
print(len(aie_all.keys()))

aie = average_indirect_effects(aie_all, relative_recovery = True)
plot_trace_heatmap(
    aie, 
    savepdf=os.path.join(save_dir, f"0_{hook}_tracing.pdf"),
    modelname="Mamba-2.8b", 
    scale_range=(0.0, 0.8) if hook == "residual" else (0.0, 0.8)
)

279
2024-03-26 13:05:45 matplotlib.colorbar DEBUG    locator: <matplotlib.ticker.AutoLocator object at 0x7fd1f4a5ec20>
2024-03-26 13:05:45 matplotlib.axes._base DEBUG    title position was updated manually, not adjusting
2024-03-26 13:05:45 matplotlib.axes._base DEBUG    title position was updated manually, not adjusting
2024-03-26 13:05:45 matplotlib.axes._base DEBUG    title position was updated manually, not adjusting
2024-03-26 13:05:45 matplotlib.backends.backend_pdf DEBUG    Assigning font /F1 = '/usr/share/fonts/truetype/msttcorefonts/Times_New_Roman.ttf'
2024-03-26 13:05:45 matplotlib.axes._base DEBUG    title position was updated manually, not adjusting
2024-03-26 13:05:45 matplotlib.backends.backend_pdf DEBUG    Embedding font /usr/share/fonts/truetype/msttcorefonts/Times_New_Roman.ttf.
2024-03-26 13:05:45 matplotlib.backends.backend_pdf DEBUG    Writing TrueType font.
2024-03-26 13:05:45 matplotlib.axes._base DEBUG    title position was updated manually, not adjusting
